In [2]:
import camelot
import pandas as pd
import openpyxl
import os
# tables = camelot.read_pdf('epd1388_2021.pdf', pages='10',process_background=True)
# tables = camelot.read_pdf('./IBU-09-20/New/14632.pdf', pages='6', split_text=True,table_areas=["69.60548391889084,500.088593765987,525.3099336281931,403.96614296894364"])
# print(len(tables))

# tables = camelot.read_pdf('./New/14699.pdf', pages='all')
# print(len(tables))

In [3]:
headers = [
    "impactGWP100_kgCO2e",
    "impactODP_kgCFC11e",
    "impactAP_kgSO2e",
    "impactEP_kgPO4e",
    "impactPOCP_kgEthenee",
    "impactADPElements_kgSbe",
    "impactADPFossilFuels_MJ",
    
    "renewablesUsedAsEnergy_MJ",
    "renewablesUsedAsMaterial_MJ",
    "embodiedRenewableEnergy_MJ",
    "nonRenewablesUsedAsEnergy_MJ",
    "nonRenewablesUsedAsMaterial_MJ",
    "embodiedEnergy_MJ",
    
    "recyclingMaterialUse_kg",
    "renewableRecylingFuelUse_MJ",
    "nonRenewableRecylingFuelUse_MJ",
    "cleanWaterNetUse_m3",
    "wasteHazardous_kg",
    "wasteNonHazardous_kg",
    "wasteRadioactive_kg",
    "reusableMaterialsOutput_kg",
    "recyclableMaterialsOutput_kg",
    "energyMaterialsOutput_kg",
    "exportedEnergyOutput_MJ_thermal",
    "exportedEnergyOutput_MJ_electric",
    "exportedEnergyOutput_MJ",
    
    "traciGWP_kgCO2e",
    "traciODP_kgCFC11e",
    "traciAP_kgSO2e",
    "traciEP_kgNe",
    "traciPOCP_kgO3e",
    "traciNRPE_MJ"
]
columns = {                                                                                                   
  "impactGWP100_kgCO2e": "(^GWP - Total$)|(^GWP$)|(^Global warming potential$)|(^Globales Erwärmungspotenzial ­ total$)|(^Globales Erwärmungspotenzial$)",
  "impactODP_kgCFC11e": "(^ODP$)|(^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)",
  "impactAP_kgSO2e": "(^AP$)|(^AP Air$)|(^Versauerungspotenzial von Boden und Wasser$)|(^Acidification potential of land and water$)",
  "impactEP_kgPO4e": "(^\*EP - Freshwater$)|(^EP$)|(^Eutrophierungspotenzial$)|(^Eutrophication potential)",
  "impactPOCP_kgEthenee": "(^POCP$)|(^Bildungspotenzial für troposphärisches Ozon$)|(^Formation potential of tropospheric ozone photochemical \noxidants)",
  "impactADPElements_kgSbe": "(^ADPE$)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nnicht fossile Ressourcen$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nnicht fossile Ressourcen$)|(^Abiotic depletion potential for non-fossil resources)",
  "impactADPFossilFuels_MJ": "(^ADPF$)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nfossile Brennstoffe$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nfossile Brennstoffe$)|(^Abiotic depletion potential for fossil resources)",
    
  "renewablesUsedAsEnergy_MJ": "(^PERE$)|(^Erneuerbare Primärenergie als Energieträger$)|(^Renewable primary energy as energy carrier$)",
  "renewablesUsedAsMaterial_MJ": "(^PERM$)|(^Erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Renewable primary energy resources as material utilization$)",
  "embodiedRenewableEnergy_MJ": "(^PERT$)|(^Total erneuerbare Primärenergie$)|(^Total use of renewable primary energy resources$)",
  "nonRenewablesUsedAsEnergy_MJ": "(^PENRE$)|(^Nicht-erneuerbare Primärenergie als Energieträger$)|(^Nicht\xaderneuerbare Primärenergie als Energieträger$)|(^Non-renewable primary energy as energy carrier$)",
  "nonRenewablesUsedAsMaterial_MJ": "(^PENRM$)|(^Nicht-erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Nicht\xaderneuerbare Primärenergie zur stofflichen Nutzung$)|(^Non-renewable primary energy as material utilization$)",
  "embodiedEnergy_MJ": "(^PENRT$)|(^Total nicht erneuerbare Primärenergie$)|(^Total use of non-renewable primary energy resources$)",
  "recyclingMaterialUse_kg": "(^SM$)|(^Einsatz von Sekundärstoffen$)|(^Use of secondary material$)",
  "renewableRecylingFuelUse_MJ": "(^RSF$)|(^Erneuerbare Sekundärbrennstoffe$)|(^Use of renewable secondary fuels$)",
  "nonRenewableRecylingFuelUse_MJ": "(^NRSF$)|(^Nicht-erneuerbare Sekundärbrennstoffe$)|(^Nicht\xaderneuerbare Sekundärbrennstoffe$)|(^Use of non-renewable secondary fuels$)",
  "cleanWaterNetUse_m3": "(^FW$)|(^Nettoeinsatz von Süßwasserressourcen$)|(^Use of net fresh water$)",
    
  "wasteHazardous_kg": "(^HWD$)|(^Gefährlicher Abfall zur Deponie$)|(^Hazardous waste disposed$)",
  "wasteNonHazardous_kg": "(^NHWD$)|(^Entsorgter nicht gefährlicher Abfall$)|(^Non-hazardous waste disposed$)",
  "wasteRadioactive_kg": "(^RWD$)|(^Entsorgter radioaktiver Abfall$)|(^Radioactive waste disposed$)",
  "reusableMaterialsOutput_kg": "(^CRU$)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)",
  "recyclableMaterialsOutput_kg": "(^MFR$)|(^Stoffe zum Recycling$)|(^Materials for recycling$)",
  "energyMaterialsOutput_kg": "(^MER$)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)",
  "exportedEnergyOutput_MJ_electric": "(^EEE$)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)",
  "exportedEnergyOutput_MJ_thermal": "(^EET$)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)",
  "exportedEnergyOutput_MJ": "(^EE$)",
    
  "traciGWP_kgCO2e": 8,
  "traciODP_kgCFC11e": 12,
  "traciAP_kgSO2e": 6,
  "traciEP_kgNe": 7,
  "traciPOCP_kgO3e": 10,
  "traciNRPE_MJ": 17
}

In [4]:
def make_excel_file(files_data):
    book = openpyxl.Workbook()
    filename = 'test_env.xlsx'
    book.save(filename)

    sheet = book['Sheet']
    sheet.cell(row=1, column=1).value = 'Filename'
    sheet.cell(row=1, column=2).value = 'EPD_Number'
    sheet.cell(row=1, column=3).value = 'DownloadLink'
    sheet.cell(row=1, column=4).value = 'Stages'
    
    count = 5
    for header in headers:
        sheet.cell(row=1, column=count).value = header
        count += 1
    
    current_row = 2
    for file_data in files_data:
        print(file_data["file_name"])
        print(file_data["type"])
        print(file_data["stages"])
        print("*******************")
        df = file_data["dataFrame"]
        base_added = False
        
#         if file_data["type"] == 1 :
        for stage in file_data["stages"]:
            if not base_added:
                sheet.cell(row=current_row, column=1).value = file_data["file_name"]
                sheet.cell(row=current_row, column=2).value = 'NO NUMBER CHECK IT!'
                sheet.cell(row=current_row, column=3).value = '/mnt/data/Downloads/2021-10-18/IBU/New/' + file_data["file_name"]
                base_added = True

            sheet.cell(row=current_row, column=4).value = stage
            header_counter = 0
            for header in headers:
                
#                 i = file_data["stages"].index(stage) + 2
#                 sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
#                 value = float(df[df[0].str.contains(columns2[header])][i].to_list()[0])
#                 #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
# #                 sheet.cell(row=current_row, column= 5 + header_counter).value = value
                try:
                    num_of_columns = len(df.columns)                    
                    i = file_data["stages"].index(stage) + (num_of_columns - len(file_data["stages"]))
                    sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
                    replaced = df[df[0].str.contains(columns[header])][i].to_list()[0].replace(',', '.')
                    if not replaced == 'IND':
                        try:
                            value = float(replaced)
                        except:
                            value = float(replaced.replace('\xad', '-'))
                    else:
                        value = replaced
                        
                    #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
                    sheet.cell(row=current_row, column= 5 + header_counter).value = value
                except:
                    print("**************************************ERRROR*****************************")
                    print(file_data["file_name"])
                    print(stage)
                    print(columns[header])
                header_counter= header_counter +1
            current_row = current_row + 1
        current_row = current_row + 1
            
    book.save(filename)
    book.close()

In [6]:
files = os.listdir('./4')
types = {}

r = []
for file in files:
    tables = camelot.read_pdf('./4/' + file, pages='all')
    print("************ FILE NAME: " + file + " ****************")
    dataFrames = []

    types[file] = []
    file_info = {}
    stages = []
    for table in tables:
        df = table.df
        stages_added = False
        #dataFrames = []
        gwp = ['GWP-total', 'GWP - Total', 'Global warming potential', 'GWP', 'Globales Erwärmungspotenzial ­ total', 'Globales Erwärmungspotenzial']
        for g in gwp:
            #if ('GWP-total' in df.values) or ('Global warming potential' in df.values) or ('GWP' in df.values) or ('Globales Erwärmungspotenzial ­ total' in df.values) or ('Globales Erwärmungspotenzial' in df.values):
            if (g in df.values) and ('TRACI 2.1 Impact Assessment' not in df.values):
                if not stages_added:
                    try:
                        #print(df.eq('Unit').any(1))
                        print(df[df.eq('Unit').any(1)])
                        print(df[df.eq('Unit').any(1)].values)
                        print(df[df.eq('Unit').any(1)].values[0])
                        print(list(df[df.eq('Unit').any(1)].values[0]))
                        a = list(df[df.eq('Unit').any(1)].values[0])
                        unit_index = a.index('Unit') + 1
                        stages = a[unit_index:]
                        print("------------STAGES--------------")
                        print(stages)
                        print(a)
                        print(unit_index)
                    except:
                        a = list(df[df.eq('Einheit').any(1)].values[0])
                        unit_index = a.index('Einheit') + 1
                        stages = a[unit_index:]
                    stages_added = True

                dataFrames.append(df)
                types[file].append(g)

        pere = ['PERE', 'Renewable primary energy as energy carrier', 'Erneuerbare Primärenergie als Energieträger']
        for p in pere:
            if p in df.values:
        #if ('PERE' in df.values) or ('Renewable primary energy as energy carrier' in df.values) or ('Erneuerbare Primärenergie als Energieträger' in df.values):
           # display(df)
                dataFrames.append(df)
                types[file].append(p)
        hwd = ['HWD', 'Hazardous waste disposed', 'Gefährlicher Abfall zur Deponie']
        for h in hwd:
        #if ('HWD' in df.values) or ('Hazardous waste disposed' in df.values) or ('Gefährlicher Abfall zur Deponie' in df.values):
            if h in df.values:
                #display(df), 
                dataFrames.append(df)
                types[file].append(h)
                
    result = pd.concat(dataFrames)
    #display(result)
    file_info["dataFrame"] = result
    if types[file] == ['GWP-total', 'PERE', 'HWD']:
        file_info["type"] = 1
    else:
        file_info["type"] = 0
        
    file_info["file_name"] = file
    file_info["stages"] = stages
    
    r.append(file_info)
    
    #write_to_excel(dataFrame, stages)
make_excel_file(r)

# print(types)
# r

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\mirol\\AppData\\Local\\Temp\\tmppdk1i5si\\page-12.png'

In [6]:
tables = camelot.read_pdf('./4/epd4042.pdf', pages='all')
print(len(tables))

4


In [16]:
tables[1].df[0][4]

'GWP - Total'

In [47]:
r[20]['file_name']

'epd4061.pdf'

In [26]:
r[0]['dataFrame'][0][4].to_list()

['GWP - Total', 'PENRE', 'CRU']

In [80]:
test_df = r[0]['dataFrame']
test_df

IndexError: list index out of range

In [49]:
a = list(test_df[test_df.eq('Unit').any(1)].values[0])
unit_index = a.index('Unit') + 1
stages = a[unit_index:]

In [51]:
len(stages)

8

In [52]:
stages

['A1-A3', 'A4', 'C1', 'C2', 'C3', 'C4', 'D', nan]

In [64]:
test_df[test_df.eq('Unit').any(1)].values

array(['Impact \nCategory', 'Unit', 'A1-A3', 'A4', 'C1', 'C2', 'C3', 'C4',
       'D', nan], dtype=object)

In [74]:


droped_nan = []
for row in test_df[test_df.eq('Unit').any(1)].values: 
    r = [e for e in row if str(e) != 'nan']
    droped_nan.append(r)
droped_nan
lengh_of_droped_nan = [len(list) for list in droped_nan]
droped_nan[lengh_of_droped_nan.index(max(lengh_of_droped_nan))]


['Impact \nCategory', 'Unit', 'A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']

In [75]:
droped_nan = []
lists = list(test_df[test_df.eq('Unit').any(1)].values)
for list in lists:
    r = [e for e in row if str(e) != 'nan']
    droped_nan.append(r)
lengh_of_droped_nan = [len(list) for list in droped_nan]
longest_list = droped_nan[lengh_of_droped_nan.index(max(lengh_of_droped_nan))]

unit_index = longest_list.index('Unit') + 1
stages = longest_list[unit_index:]

In [76]:
stages

['A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']

In [ ]:
droped_nan = []
lists = list(df[df.eq('Unit').any(1)].values)
for list in lists:
    r = [e for e in list if str(e) != 'nan']
    droped_nan.append(r)
lengh_of_droped_nan = [len(list) for list in droped_nan]
longest_list = droped_nan[lengh_of_droped_nan.index(max(lengh_of_droped_nan))]

unit_index = longest_list.index('Unit') + 1
stages = longest_list[unit_index:]

In [10]:
r

['Impact \nCategory',
 'Unit',
 'A1-A3',
 'A4',
 'A5',
 'C1',
 'C2',
 'C3',
 'C4',
 'D',
 {'dataFrame':                     0                                                  1  \
  0   Impact \nCategory                                               Unit   
  1        GWP - Fossil                                          kg CO2 eq   
  2      GWP - Biogenic                                          kg CO2 eq   
  3         GWP - Luluc                                          kg CO2 eq   
  4         GWP - Total                                          kg CO2 eq   
  5                 ODP                                       kg CFC-11 eq   
  6                  AP                                          mol H+ eq   
  7     EP - Freshwater                                            kg P eq   
  8    *EP - Freshwater                                          kg PO4 eq   
  9         EP - Marine                                            kg N eq   
  10   EP - Terrestrial                 

In [7]:
r

[]